# Module Import Cell

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

## Data Import Cell

In [3]:
#Available population density by state and year meassured in persons per square mile
pop_df = pd.read_csv('data/Historical_Population_Density_States_only_RAND_US_1.csv')
#---------------------------------------------------------------------------------------
#Available firearm background check data
nics_df = pd.read_csv('data/nics-firearm-background-checks.csv')

In [4]:
nics_df.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2020-02,Alabama,29633.0,604.0,24590.0,12531.0,1225.0,1217,0.0,45.0,...,0.0,0.0,0.0,32.0,17.0,3.0,0.0,3.0,0.0,75336
1,2020-02,Alaska,139.0,10.0,2560.0,1839.0,311.0,203,0.0,0.0,...,0.0,0.0,0.0,4.0,9.0,0.0,0.0,0.0,0.0,5466
2,2020-02,Arizona,6138.0,741.0,16839.0,8006.0,1424.0,1392,0.0,20.0,...,0.0,0.0,0.0,9.0,7.0,2.0,1.0,0.0,0.0,37100
3,2020-02,Arkansas,2994.0,731.0,8112.0,5453.0,363.0,542,4.0,15.0,...,0.0,0.0,0.0,18.0,8.0,4.0,0.0,0.0,0.0,21445
4,2020-02,California,32002.0,0.0,35570.0,22645.0,4221.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95430


In [33]:
nics_df.tail(20)

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
14060,1998-11,North Carolina,524.0,NaN,87.0,695.0,NaN,4,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1310
14061,1998-11,North Dakota,0.0,NaN,20.0,38.0,NaN,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58
14062,1998-11,Ohio,0.0,NaN,502.0,434.0,NaN,16,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,952
14063,1998-11,Oklahoma,0.0,NaN,259.0,361.0,NaN,13,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,633
14064,1998-11,Oregon,0.0,NaN,153.0,186.0,NaN,5,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,344
14065,1998-11,Pennsylvania,0.0,NaN,5.0,8.0,NaN,4,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
14066,1998-11,Puerto Rico,0.0,NaN,1.0,1.0,NaN,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
14067,1998-11,Rhode Island,0.0,NaN,13.0,23.0,NaN,2,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
14068,1998-11,South Carolina,0.0,NaN,0.0,6.0,NaN,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
14069,1998-11,South Dakota,0.0,NaN,8.0,66.0,NaN,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74


In [27]:
pop_df.head()

,Area,Units,1790,1791,1792,1793,1794,1795,1796,1797,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Alabama,Persons per square mile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.79339,94.30569,94.56949,94.89918,95.19273,95.42963,95.64008,95.86838,96.07284,96.32412
1,Alaska,Persons per square mile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.22121,1.24819,1.26241,1.27703,1.28865,1.28736,1.28953,1.29645,1.29344,1.28934
2,Arizona,Persons per square mile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.04376,56.38909,56.96746,57.69903,58.38869,59.25850,60.13637,61.12071,62.03085,63.11124
3,Arkansas,Persons per square mile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.49360,56.11850,56.47244,56.69718,56.84007,56.99712,57.20225,57.43278,57.67452,57.88248
4,California,Persons per square mile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,236.99551,239.29945,241.35717,243.40232,245.45441,247.66214,249.76527,251.40663,252.62632,253.63746


In [34]:
# cleaner_pop_df = pop_df.dropna(axis=1)
# cleaner_pop_df = cleaner_pop_df.loc[:, ['Area'] + list(map(str, range(1998, 2019)))]
# cleaner_pop_df = cleaner_pop_df.rename(columns={'Area': 'State'})
# cleaner_pop_df.head()

,State,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Alabama,85.74486,86.11584,87.73153,87.97166,88.13692,88.49501,88.92066,89.56821,90.60555,...,92.79339,94.30569,94.56949,94.89918,95.19273,95.42963,95.64008,95.86838,96.07284,96.32412
1,Alaska,1.07562,1.08313,1.09712,1.10729,1.12368,1.13801,1.15669,1.17053,1.18424,...,1.22121,1.24819,1.26241,1.27703,1.28865,1.28736,1.28953,1.29645,1.29344,1.28934
2,Arizona,41.07269,42.04999,45.46765,46.67961,47.97931,49.20339,50.68374,52.57937,54.49134,...,58.04376,56.38909,56.96746,57.69903,58.38869,59.25850,60.13637,61.12071,62.03085,63.11124
3,Arkansas,48.74767,49.00063,51.43811,51.68355,51.94598,52.28321,52.74165,53.31897,54.06560,...,55.49360,56.11850,56.47244,56.69718,56.84007,56.99712,57.20225,57.43278,57.67452,57.88248
4,California,209.55971,212.52412,217.97073,221.11932,223.62363,226.02755,227.99801,229.51658,230.69608,...,236.99551,239.29945,241.35717,243.40232,245.45441,247.66214,249.76527,251.40663,252.62632,253.63746


In [25]:
# cleaner_pop_df.shape

(52, 22)